<a href="https://colab.research.google.com/github/buvnswrn/CNTG-Project/blob/main/Reconstruction_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconstruction Model

Part of the Pretraining for the DAML-ST5 Model: We need to contruct a model that reconstructs the given sentence

Deployement URL: https://huggingface.co/buvnswrn/daml-t5-pretrain-imdb


## Check Hardware Details

Verifying which GPU is allocated

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Apr  1 19:25:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Verifying the runtime type (RAM)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


## Check Pre-requisites

In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     |████████████████████████████████| 325 kB 8.5 MB/s 
     |████████████████████████████████| 3.8 MB 56.8 MB/s 
     |████████████████████████████████| 67 kB 6.6 MB/s 
     |████████████████████████████████| 136 kB 79.5 MB/s 
     |████████████████████████████████| 212 kB 75.5 MB/s 
     |████████████████████████████████| 1.1 MB 58.2 MB/s 
     |████████████████████████████████| 127 kB 67.5 MB/s 
     |████████████████████████████████| 271 kB 66.4 MB/s 
     |████████████████████████████████| 94 kB 739 kB/s 
     |████████████████████████████████| 144 kB 89.6 MB/s 
     |████████████████████████████████| 596 kB 86.4 MB/s 
     |████████████████████████████████| 6.5 MB 51.5 MB/s 
     |████████████████████████████████| 895 kB 60.0 MB/s 
     |████████████████████████████████| 1.2 MB 80.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
   

Setting up Git

In [ ]:
!git config --global user.email "bhuvan.nav@gmail.com"
!git config --global user.name "buvnswrn"

Login to Hugging face hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Dataset

Load the IMDB Dataset

In [ ]:
from datasets import load_dataset, load_metric

# raw_datasets = load_dataset("imdb")
raw_datasets = load_dataset("yelp_polarity")
# raw_datasets = load_dataset("amazon_polarity")

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/a770787b2526bdcbfc29ac2d9beb8e820fbc15a03afd3ebc4fb9d8529de57544. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

### Test the dataset

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 38000
    })
})

In [ ]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 504000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 56000
    })
})

In [ ]:
split_datasets["validation"] = split_datasets.pop("test")

In [ ]:
split_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 504000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 56000
    })
})

In [ ]:
split_datasets["train"][1]["text"]

"I'm sucker for Asian food!\\n\\nMars says he wanna go eat Chinese food....My eyes opened up as if I saw the titanic sinking.. Hell yeah I want Chinese food!  she said it's cash only.. I was like even better most the experience from Cash only is the shizzle! \\n\\nWe were staying on the strip, so the distance was about 10 min... Max\\n\\nWe sat down didn't get any service..finally after 10 min we received water, and we order. food came out pretty fast. \\n\\nIt was OK! we ordered usual dishes chicken/broccoli, beef chow mein, and some other noddles. They are very cheap with their meats. I felt I was caveman I had kill for a piece of meat. \\n\\nSo, next time I'm in LV, and I want authentic Chinese I'll yelp for somewhere else."

In [ ]:
split_datasets["train"][172]["text"]

'The witch doctor clenched three small dolls in his gnarled fist. While they looked nothing like me or my friends they had been carefully doused in our favorite liquor, had hairs on top surreptitiously plucked from our heads and were clothed in socks, missing from our last visit to Vegas. The witch doctor cackled with maniacal laughter knowing he would royally kill our buzzes in Vegas.\\n\\nThe torture began by throwing all his victims\' dolls together, bundled tightly with twine for over an hour and a half. He planned his night of nefariousness while dining on gruel of enemies\' bones. At the same time my friends and I were in painful limbo, outside Body English, packed like sardines, begging to be let in. Little did we know we were part of a sadist\'s sick game. http://www.yelp.com/biz/GCRvrxMSC1nzShyM4Y-guQ?hrid=mYQzT_gkw5feEE4GSNMszg\\n\\nThe witch doctor snorted to announce the end of our wait. He raised us and other dolls high above his head - reciting the most evil of incantatio

## Reconstruction Model

**Load the T5 Base Model translation pipeline**

We will consider the reconstruction task as translating English to English. Since we do not include any prefix, the model will be training for default translation task - Translating from English to German. We can check that by translating a sentence before training.

In [ ]:
from transformers import pipeline

model_checkpoint = "buvnswrn/daml-t5-pretrain-imdb"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/__init__.py:654: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  UserWarning,


[{'translation_text': 'translate English to German: Default to expanded threads'}]

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

[{'translation_text': 'translate English to German: Unable to import %1 using the OFX importer plugin. This file is not the correct format.'}]

**Use the pretrained tokenizer**

In [ ]:
from transformers import AutoTokenizer

tokenizer_checkpoint = "buvnswrn/t5-base-yelp-tokenizer"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint, return_tensors="tf")

Downloading:   0%|          | 0.00/1.91k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

#### Test the tokenizer [No need to run this]

In [ ]:
en_sentence = split_datasets["train"][1]["text"]
fr_sentence = split_datasets["train"][1]["text"]

inputs = tokenizer(en_sentence)
with tokenizer.as_target_tokenizer():
    targets = tokenizer(fr_sentence)

In [ ]:
wrong_targets = tokenizer(fr_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(targets["input_ids"]))

["▁I'm", '▁sucker', '▁for', '▁Asian', '▁food', '!\\n\\n', 'Mar', 's', '▁says', '▁he', '▁wanna', '▁go', '▁eat', '▁Chinese', '▁food.', '...', 'My', '▁eyes', '▁opened', '▁up', '▁as', '▁if', '▁I', '▁saw', '▁the', '▁', 't', 'it', 'a', 'nic', '▁sink', 'ing.', '.', '▁Hell', '▁yeah', '▁I', '▁want', '▁Chinese', '▁food!', '▁she', '▁said', "▁it's", '▁cash', '▁only', '.', '.', '▁I', '▁was', '▁like', '▁even', '▁better', '▁most', '▁the', '▁experience', '▁from', '▁Cash', '▁only', '▁is', '▁the', '▁sh', 'izzle', '!', '▁\\n\\nWe', '▁were', '▁staying', '▁on', '▁the', '▁strip,', '▁so', '▁the', '▁distance', '▁was', '▁about', '▁10', '▁min', '...', '▁Max', '\\n\\nWe', '▁sat', '▁down', "▁didn't", '▁get', '▁any', '▁service.', '.', 'finally', '▁after', '▁10', '▁min', '▁we', '▁received', '▁water,', '▁and', '▁we', '▁order.', '▁food', '▁came', '▁out', '▁pretty', '▁fast.', '▁\\n\\nI', 't', '▁was', '▁OK', '!', '▁we', '▁ordered', '▁usual', '▁dishes', '▁chicken', '/', 'broccoli', ',', '▁beef', '▁chow', '▁mein', ',', '

### Preprocessing Function

In [ ]:
max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = [ex for ex in examples["text"]]
    targets = [ex for ex in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Set up the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    # model_inputs["prefix"] = prefix
    return model_inputs

### Tokenize the dataset

In [ ]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

  0%|          | 0/504 [00:00<?, ?ba/s]

  0%|          | 0/56 [00:00<?, ?ba/s]

### Fetch the model from Hugginface Hub

In [ ]:
from transformers import AutoModelWithLMHead
model_checkpoint = "buvnswrn/daml-t5-pretrain-imdb"
model = AutoModelWithLMHead.from_pretrained(model_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


### Form Batch using Datacollator

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#### Test DataCollator [No need to run this]

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [ ]:
batch["labels"]

tensor([[  210,  5834,   113,  1294,   141,  1498, 10573,   124,   926,   186,
          3862,   161,   282,   894,   378,   174,  8543,  1818,   990,   154,
           139,   155,   106,   557,   103,   112,   198,   871,   216,  6355,
          3503,  2612,   109,  5603,  3697,   106,   246,   894,  2348,   192,
           226,   203,  1209,   172,   109,   109,   106,   110,   140,   189,
           251,   317,   103,   309,   151, 12647,   172,   115,   103,  3622,
         16684,   136,   805,   131,  1026,   120,   103,  2982,   135,   103,
          2848,   110,   156,   430,  1166,   174,  7664,  5000,   585,   270,
           197,   144,   245,   453,   109, 17387,   215,   430,  1166,   130,
           745,  3010,   104,   130,   877,   141,   213,   143,   242,  3082,
           334,   198,   110,  1258,   136,   130,   207,  1440,   633,   281,
           300, 24635,   108,   576,  4455,  4723,   108,   104,   169,   191,
           170, 10124,   303,   188,   134,   148,  

In [ ]:
batch["decoder_input_ids"]

tensor([[    0,   210,  5834,   113,  1294,   141,  1498, 10573,   124,   926,
           186,  3862,   161,   282,   894,   378,   174,  8543,  1818,   990,
           154,   139,   155,   106,   557,   103,   112,   198,   871,   216,
          6355,  3503,  2612,   109,  5603,  3697,   106,   246,   894,  2348,
           192,   226,   203,  1209,   172,   109,   109,   106,   110,   140,
           189,   251,   317,   103,   309,   151, 12647,   172,   115,   103,
          3622, 16684,   136,   805,   131,  1026,   120,   103,  2982,   135,
           103,  2848,   110,   156,   430,  1166,   174,  7664,  5000,   585,
           270,   197,   144,   245,   453,   109, 17387,   215,   430,  1166,
           130,   745,  3010,   104,   130,   877,   141,   213,   143,   242,
          3082,   334,   198,   110,  1258,   136,   130,   207,  1440,   633,
           281,   300, 24635,   108,   576,  4455,  4723,   108,   104,   169,
           191,   170, 10124,   303,   188,   134,  

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[210, 5834, 113, 1294, 141, 1498, 10573, 124, 926, 186, 3862, 161, 282, 894, 378, 174, 8543, 1818, 990, 154, 139, 155, 106, 557, 103, 112, 198, 871, 216, 6355, 3503, 2612, 109, 5603, 3697, 106, 246, 894, 2348, 192, 226, 203, 1209, 172, 109, 109, 106, 110, 140, 189, 251, 317, 103, 309, 151, 12647, 172, 115, 103, 3622, 16684, 136, 805, 131, 1026, 120, 103, 2982, 135, 103, 2848, 110, 156, 430, 1166, 174, 7664, 5000, 585, 270, 197, 144, 245, 453, 109, 17387, 215, 430, 1166, 130, 745, 3010, 104, 130, 877, 141, 213, 143, 242, 3082, 334, 198, 110, 1258, 136, 130, 207, 1440, 633, 281, 300, 24635, 108, 576, 4455, 4723, 108, 104, 169, 191, 170, 10124, 303, 188, 134, 148, 706, 1]
[106, 902, 143, 125, 3904, 120, 868, 1806, 107, 1332, 195, 139, 106, 389, 195, 1172, 104, 110, 2094, 114, 105, 1981, 128, 129, 2746, 120, 2933, 12908, 109, 106, 835, 103, 868, 554, 174, 169, 975, 104, 169, 785, 104, 386, 106, 325, 237, 105, 492, 107, 262, 103, 1981, 189, 408, 128, 129, 169, 242, 299, 554, 120, 160, 354, 

## Evaluation

For the evaluation of the Reconstructor model, we use `sacrebleu` library for computing the BLEU scores.

### Prerequisites

In [ ]:
!pip install sacrebleu

     |████████████████████████████████| 90 kB 5.2 MB/s 


### Load BLEU Metric

In [ ]:
from datasets import load_metric

metric = load_metric("sacrebleu")

#### Test BLEU Metric [No need to run this]

In [ ]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'bp': 0.9200444146293233,
 'counts': [11, 6, 4, 3],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'ref_len': 13,
 'score': 46.750469682990165,
 'sys_len': 12,
 'totals': [12, 11, 10, 9]}

In [ ]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'bp': 0.10539922456186433,
 'counts': [1, 0, 0, 0],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'ref_len': 13,
 'score': 1.683602693167689,
 'sys_len': 4,
 'totals': [4, 3, 2, 1]}

In [ ]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'bp': 0.004086771438464067,
 'counts': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'ref_len': 13,
 'score': 0.0,
 'sys_len': 2,
 'totals': [2, 1, 0, 0]}

### Function to compute metrics

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

#### Login to Huggingface hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


## Training the Model

Don't use this method is Custom Training Loop is used

Create training arguments for the trainer object

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"daml-t5-pretrain-yelp",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

Creating an instance of the sequence to sequence trainer

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

#### Test the trainer

In [ ]:
trainer.evaluate(max_length=max_target_length)

***** Running Evaluation *****
  Num examples = 56000
  Batch size = 64


{'eval_bleu': 99.10211114167431,
 'eval_loss': 0.0024576298892498016,
 'eval_runtime': 2716.5202,
 'eval_samples_per_second': 20.615,
 'eval_steps_per_second': 0.322}

We coud notice that the evaluation gives very less BLEU scores since the model is not trained

### Start Training

Starting the training with the default number of epochs

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 504000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 47250


Step,Training Loss
500,0.006900
1000,0.004800
1500,0.003800
2000,0.003300
2500,0.002700
3000,0.002300
3500,0.002300
4000,0.001900
4500,0.001600
5000,0.001600


Saving model checkpoint to daml-t5-pretrain-yelp/checkpoint-15750
Configuration saved in daml-t5-pretrain-yelp/checkpoint-15750/config.json
Model weights saved in daml-t5-pretrain-yelp/checkpoint-15750/pytorch_model.bin
tokenizer config file saved in daml-t5-pretrain-yelp/checkpoint-15750/tokenizer_config.json
Special tokens file saved in daml-t5-pretrain-yelp/checkpoint-15750/special_tokens_map.json
tokenizer config file saved in daml-t5-pretrain-yelp/tokenizer_config.json
Special tokens file saved in daml-t5-pretrain-yelp/special_tokens_map.json


### Save to Hub

Save the trained model to the huggingface hub

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete for T5 Pretraining on Yelp")

### Evaluate the trained model

In [ ]:
trainer.evaluate(max_length=max_target_length)

## Custom Training Loop (Not using builtin method to train)

Deployment URL: https://huggingface.co/buvnswrn/daml-t5-pretrain-imdb

In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [ ]:
model = AutoModelWithLMHead.from_pretrained(model_checkpoint)

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:882: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file https://huggingface.co/buvnswrn/daml-t5-pretrain-imdb/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/7a26db371e624209d8f6f279c068f2fb4e544cef57ce60c478ee0fe378f93c79.b3a62b41aa10480c55fcd380771e43aeef76ca27504ebd270d4ee63af528f521
Model config T5Config {
  "_name_or_path": "buvnswrn/daml-t5-pretrain-imdb",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decode

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader # add other model as well here
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "daml-t5-pretrain-imdb"
repo_name = get_full_repo_name(model_name)
repo_name

'buvnswrn/daml-t5-pretrain-imdb'

In [ ]:
output_dir = "daml-t5-pretrain-imdb-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

/content/daml-t5-pretrain-imdb-accelerate is already a clone of https://huggingface.co/buvnswrn/daml-t5-pretrain-imdb. Make sure you pull the latest changes with `repo.git_pull()`.


### Post-Process Function

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
batch

{'input_ids': tensor([[ 5046, 20334,   305, 11263,   115,   185,   140,  1142,   334,   129,
           166, 16157,   111,   105,   631,  5449,   137,   117,   152,   237,
           118,   726,   107,  3187,   142,  4049,   104,   125,   110,   103,
           137, 11837,   124,   142,   105,   268,  5499,   240,   103,   790,
          7057,   111,  7137,  2681,   108,   135,   116,   580,   133,   855,
           107,  1273,   108,   121,   399,  1480,   135,   340,   103,   542,
          5499,   109,  7178,   122,   168,   297,   103,   439,   111,   112,
          8515,   500,   119,   142,   156,   363,  5449,   413,   106,  1614,
           109,   123,  1072,   748,   103,   775,   211,   127,  2070,   103,
           499,   162,   104,   114,   590,  1333,   388,   132,   138,   140,
           106,   580,   133, 11682,   212,   108,   121,   192,  1039,   107,
           103,  5259,   111,   103,   397,  2622,   134,   147,   206,   103,
           137,   151,  1144,   113,  

In [ ]:
# model.train()
for batch in train_dataloader:
  for i in range(0,len(batch["input_ids"])):
    outputs = model(input_ids=batch["input_ids"][i].unsqueeze(0),
                    labels=batch["labels"][i].unsqueeze(0))
    print(outputs.loss)
  break

tensor(0.0861, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0005, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0496, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1757, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0865, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.1650, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(8.7407e-05, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(1.3118, device='cuda:0', grad_fn=<NllLossBackward0>)


In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        # add discriminator
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

  0%|          | 0/8439 [00:00<?, ?it/s]

  0%|          | 0/313 [00:00<?, ?it/s]

Configuration saved in daml-t5-pretrain-imdb-accelerate/config.json


epoch 0, BLEU score: 99.96


Model weights saved in daml-t5-pretrain-imdb-accelerate/pytorch_model.bin
tokenizer config file saved in daml-t5-pretrain-imdb-accelerate/tokenizer_config.json
Special tokens file saved in daml-t5-pretrain-imdb-accelerate/special_tokens_map.json


...

Model weights saved in daml-t5-pretrain-imdb-accelerate/pytorch_model.bin
tokenizer config file saved in daml-t5-pretrain-imdb-accelerate/tokenizer_config.json
Special tokens file saved in daml-t5-pretrain-imdb-accelerate/special_tokens_map.json
Several commits (2) will be pushed upstream.


  0%|          | 0/313 [00:00<?, ?it/s]

Configuration saved in daml-t5-pretrain-imdb-accelerate/config.json


epoch 2, BLEU score: 99.99


Model weights saved in daml-t5-pretrain-imdb-accelerate/pytorch_model.bin
tokenizer config file saved in daml-t5-pretrain-imdb-accelerate/tokenizer_config.json
Special tokens file saved in daml-t5-pretrain-imdb-accelerate/special_tokens_map.json
Several commits (3) will be pushed upstream.


### Test the model

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "daml-t5-pretrain-imdb-accelerate"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

loading configuration file daml-t5-pretrain-imdb-accelerate/config.json
Model config T5Config {
  "_name_or_path": "daml-t5-pretrain-imdb-accelerate",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
      "num_beam

[{'translation_text': 'translate English to German: Default to expanded threads'}]

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

[{'translation_text': 'translate English to German: Unable to import %1 using the OFX importer plugin. This file is not the correct format.'}]